In [27]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

# Read in Data

In [28]:
%run "Data Collection and Cleaning.ipynb"

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [29]:
#create dataframe with the high correlation columns, and split up into train and test data
high_corr_df = df_merged.copy()
high_corr_df = high_corr_df[high_corr_cols]

y_data = df_merged["W"].values.reshape(-1, 1)
x_data = df_merged[high_corr_cols].set_index(['abbreviation', 'Season'])


x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, random_state = 42)

In [30]:
#Train Linear Rregression Model
regr = LinearRegression().fit(x_train, y_train)

y_predict_train = regr.predict(x_train)
y_predict_test = regr.predict(x_test)

Since sklearn accuracy function is based on exact matches, it is tough to use the accuracy_score function since it will compare a decimal to an integer. So we will test in two different ways. The first test will be by rounding to the nearest integer, and comparing that result using the accuracy score. The seconcd test will be to get the rate of wins within X of the true value, where X is user defined.

Test #1

In [31]:
def rounded_accuracy(y_actual, y_pred):
    round_y_predict = []

    for y in y_pred:
        rounded_y = round(float(y))
        round_y_predict.append(rounded_y)
        
    round_y_predict_arr = np.array(round_y_predict)
    accuracy_sc = accuracy_score(y_actual, round_y_predict_arr)

    return accuracy_sc

In [32]:
train_accuracy = rounded_accuracy(y_train,y_predict_train)
print("Training Accuracy based on rounded: ",train_accuracy)

test_accuracy = rounded_accuracy(y_test,y_predict_test)
print("Test Accuracy based on rounded: ",test_accuracy)

Training Accuracy based on rounded:  0.44776119402985076
Test Accuracy based on rounded:  0.4782608695652174


Test #2

In [33]:
def calc_accuracy_rate(match_bool):
    true_count = 0
    total_matches = len(match_bool)
    for match in match_bool:
        if match:
            true_count += 1

    return true_count / total_matches

In [34]:
#Read linear Regression Model's Accuracy
def get_accuracy(num_wins_window, y_predict, y_actual):
    # if number of wins predicted is within 5 of the actual number of wins 
    # then this will be marked as a "success"
    # (for example if actual number of wins is 65, "correct window" is 60-70)
    # this can easily be adjusted based on the users desired level of accuracy

    round_y_predict = []

    for i,y in enumerate(y_predict):
        wins_diff = abs(y - y_actual[i])
        if wins_diff <= num_wins_window:
            match = True
        else:
            match = False
        round_y_predict.append(match)

        
    # round_y_predict_arr = np.array(round_y_predict)
    train_accuracy = calc_accuracy_rate(round_y_predict)

    return train_accuracy

In [35]:
train_accuracy_0 = get_accuracy(0,y_predict_train,y_train)
print("Exact Accuracy:",train_accuracy_0)

train_accuracy_1 = get_accuracy(1,y_predict_train,y_train)
print("Rate of correct wins within 1:", train_accuracy_1)

Exact Accuracy: 0.16417910447761194
Rate of correct wins within 1: 1.0


In [36]:
test_accuracy_0 = get_accuracy(0,y_predict_test,y_test)
print("Exact Accuracy:",test_accuracy_0)

test_accuracy_1 = get_accuracy(1,y_predict_test,y_test)
print("Rate of correct wins within 1:", test_accuracy_1)


Exact Accuracy: 0.13043478260869565
Rate of correct wins within 1: 1.0


With the High Correlation variables, all games are correctly predicted within one win. This is for both Test and Training datasets